# Creating, saving and loading datasets with CSBMs

In [2]:
import torch

from csbm import MultiClassCSBM

In [2]:
torch.set_printoptions(precision=3)
csbm = MultiClassCSBM(n=8, classes=2, dimensions=4)
data_list = [csbm.data]
for i in range(4):
    csbm.evolve()
    data_list.append(csbm.data)

## Simply by hand